In [1]:
import time
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName('assignment-1').getOrCreate()

In [3]:
rdd = spark.sparkContext.textFile('wikipedia.dat')

In [4]:
search_words = ['JavaScript','Java','PHP','Python','C#','C++','Ruby','CSS','C','Objective-C','Shell','R','Pearl','Scala',
                'Go','Haskell','Swift','Matlab','Visual Basic','Clojure','Groovy']

search_words_lower = ['javascript','java','php','python','c#','c++','ruby','css','c','objective-c','shell','r','pearl','scala',
                'haskell','swift','matlab','visual basic','clojure','groovy']



occurrences_rdd = rdd.flatMap(lambda line: line.split(" ")).filter(lambda word: word in search_words).map(lambda word: (word, 1))
occurrences_rdd_lower_1 = rdd.flatMap(lambda line: line.split(" ")).filter(lambda word: word in search_words_lower).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)



In [5]:
start = time.time()

occurrences_rdd = occurrences_rdd.reduceByKey(lambda a, b: a + b)
occurrences_rdd_lower_1 = occurrences_rdd_lower_1.reduceByKey(lambda a, b: a + b)

print(time.time() - start)

0.03999805450439453


In [6]:
occurrences_rdd_lower_2 = occurrences_rdd.map(lambda x: (x[0].lower(),x[1]))

In [7]:
final_rdd = occurrences_rdd_lower_2.join(occurrences_rdd_lower_1)

In [8]:
result_rdd = final_rdd.mapValues(lambda x: sum([i for i in x if i is not None]))

In [9]:
sort_rdd = result_rdd.map(lambda x: (x[1],x[0])).sortByKey(ascending=False)
sort_rdd.collect()

[(5190, 'javascript'),
 (2670, 'java'),
 (2223, 'css'),
 (2112, 'c'),
 (1666, 'c#'),
 (1255, 'c++'),
 (882, 'php'),
 (835, 'python'),
 (688, 'r'),
 (348, 'objective-c'),
 (329, 'ruby'),
 (308, 'shell'),
 (147, 'haskell'),
 (124, 'matlab'),
 (81, 'scala'),
 (76, 'swift'),
 (35, 'groovy'),
 (23, 'pearl')]

In [10]:
df_rdd = sort_rdd.map(lambda x: (x[1],x[0])).toDF(["Programing Language","Occurences"])

In [11]:
df_rdd.show()

+-------------------+----------+
|Programing Language|Occurences|
+-------------------+----------+
|         javascript|      5190|
|               java|      2670|
|                css|      2223|
|                  c|      2112|
|                 c#|      1666|
|                c++|      1255|
|                php|       882|
|             python|       835|
|                  r|       688|
|        objective-c|       348|
|               ruby|       329|
|              shell|       308|
|            haskell|       147|
|             matlab|       124|
|              scala|        81|
|              swift|        76|
|             groovy|        35|
|              pearl|        23|
+-------------------+----------+

